In [310]:
from matplotlib import pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import pandas as pd
import xarray as xr
from morpher.config import parse
import zarr
import fsspec
import xclim as xc


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

In [255]:
models = pd.read_csv(parse('modelsources'))
models = models[models['in_ensemble']=='Yes']['source_id'].tolist()

['ACCESS-CM2', 'BCC-CSM2-MR', 'CanESM5', 'GFDL-CM4', 'INM-CM5-0', 'TaiESM1']

In [58]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.sample(6)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
123883,ScenarioMIP,CCCma,CanESM5,ssp126,r21i1p2f1,Omon,spco2abio,gn,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...,NaN,20190429
105678,ScenarioMIP,CCCma,CanESM5,ssp585,r5i1p2f1,Amon,rlut,gn,gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp...,NaN,20190429
156535,DCPP,CCCma,CanESM5,dcppA-hindcast,r6i1p2f1,Amon,hus,gn,gs://cmip6/DCPP/CCCma/CanESM5/dcppA-hindcast/s...,2005.0,20190429
185918,DCPP,CCCma,CanESM5,dcppA-hindcast,r11i1p2f1,Amon,huss,gn,gs://cmip6/DCPP/CCCma/CanESM5/dcppA-hindcast/s...,1974.0,20190429
241468,ScenarioMIP,DKRZ,MPI-ESM1-2-HR,ssp126,r1i1p1f1,Oyr,ppdiaz,gn,gs://cmip6/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-H...,NaN,20190710
61410,ScenarioMIP,BCC,BCC-CSM2-MR,ssp585,r1i1p1f1,Lmon,tran,gn,gs://cmip6/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/s...,NaN,20190308


In [59]:
df['activity_id'].unique()

array(['HighResMIP', 'CMIP', 'CFMIP', 'ScenarioMIP', 'RFMIP',
       'AerChemMIP', 'LUMIP', 'DAMIP', 'FAFMIP', 'OMIP', 'GMMIP', 'C4MIP',
       'CDRMIP', 'PMIP', 'LS3MIP', 'DCPP', 'PAMIP', 'ISMIP6'],
      dtype=object)

In [ ]:
df_ta = df.query(f"activity_id=='ScenarioMIP' & table_id == 'Amon' & variable_id == 'tas' & experiment_id == 'ssp126' & member_id == 'r1i1p1f1'")
scen_links = df_ta[df_ta['source_id'].isin(models)]
scen_links

In [254]:
insts = scen_links['institution_id'].tolist()
insts

['BCC', 'CCCma', 'INM', 'AS-RCEC', 'CSIRO-ARCCSS']

In [69]:
ds_dict = dict()

for inst in insts:
    print(inst)
    df_ta = scen_links.query(f"institution_id == '{inst}'")

    # get the path to a specific zarr store (the first one from the dataframe above)
    zstore = df_ta.zstore.values[-1]
    print(zstore)

    # create a mutable-mapping-style interface to the store
    mapper = fsspec.get_mapper(zstore)

    # open it using xarray and zarr
    ds = xr.open_zarr(mapper, consolidated=True)
    ds_dict[inst] = ds

    print(f"Lat min is {round(ds.lat.min().values.tolist(),2)} and \n lat max is {round(ds.lat.max().values.tolist(),2)}")
    print(f"Lon min is {ds.lon.min().values.tolist()} and \n lon max is {ds.lon.max().values.tolist()}")

BCC
gs://cmip6/CMIP6/ScenarioMIP/BCC/BCC-CSM2-MR/ssp126/r1i1p1f1/Amon/tas/gn/v20190314/


Lat min is -89.14 and 
 lat max is 89.14
Lon min is 0.0 and 
 lon max is 358.875
CCCma
gs://cmip6/CMIP6/ScenarioMIP/CCCma/CanESM5/ssp126/r1i1p1f1/Amon/tas/gn/v20190429/
Lat min is -87.86 and 
 lat max is 87.86
Lon min is 0.0 and 
 lon max is 357.1875
INM
gs://cmip6/CMIP6/ScenarioMIP/INM/INM-CM5-0/ssp126/r1i1p1f1/Amon/tas/gr1/v20190619/
Lat min is -89.25 and 
 lat max is 89.25
Lon min is 0.0 and 
 lon max is 358.0
AS-RCEC
gs://cmip6/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/ssp126/r1i1p1f1/Amon/tas/gn/v20201124/
Lat min is -90.0 and 
 lat max is 90.0
Lon min is 0.0 and 
 lon max is 358.75
CSIRO-ARCCSS
gs://cmip6/CMIP6/ScenarioMIP/CSIRO-ARCCSS/ACCESS-CM2/ssp126/r1i1p1f1/Amon/tas/gn/v20210317/
Lat min is -89.38 and 
 lat max is 89.38
Lon min is 0.9375 and 
 lon max is 359.0625


/opt/homebrew/Caskroom/miniforge/base/envs/epwmorpher/lib/python3.9/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/opt/homebrew/Caskroom/miniforge/base/envs/epwmorpher/lib/python3.9/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/opt/homebrew/Caskroom/miniforge/base/envs/epwmorpher/lib/python3.9/site-packages/numpy/core/_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=Fa

In [108]:
a = ds_dict[insts[0]].tas
b = ds_dict[insts[1]].tas

all_models = xr.concat([a.assign_coords({'models': insts[0]}),
                        b.assign_coords({'models': insts[1]})],dim='models')

/opt/homebrew/Caskroom/miniforge/base/envs/epwmorpher/lib/python3.9/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/opt/homebrew/Caskroom/miniforge/base/envs/epwmorpher/lib/python3.9/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunk

In [133]:
mod = insts[5]
ds_dict[mod].tas.assign_coords({'models': mod}).time

IndexError: list index out of range

In [161]:
arrays = []

for n in insts:
    if n=='CSIRO-ARCCSS':
        pass
    else:
        print(n)
        da = ds_dict[n].tas.assign_coords({'models': n})
        da = da.sel(time=slice('2015', '2100'))
        arrays.append(da)

BCC
CCCma
INM
AS-RCEC


/opt/homebrew/Caskroom/miniforge/base/envs/epwmorpher/lib/python3.9/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/opt/homebrew/Caskroom/miniforge/base/envs/epwmorpher/lib/python3.9/site-packages/xarray/core/indexing.py:1233: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunk

<xarray.Dataset>
Dimensions:      (lat: 536, lon: 768, realization: 4, time: 1032)
Coordinates:
  * lat          (lat) float64 -90.0 -89.25 -89.14 -89.06 ... 89.14 89.25 90.0
  * lon          (lon) float64 0.0 1.125 1.25 2.0 ... 357.8 358.0 358.8 358.9
    height       float64 2.0
  * time         (time) datetime64[ns] 2015-01-16T12:00:00 ... 2100-12-16T12:...
    models       (realization) <U7 'BCC' 'CCCma' 'INM' 'AS-RCEC'
  * realization  (realization) int64 0 1 2 3
Data variables:
    tas          (realization, time, lat, lon) float32 dask.array<chunksize=(1, 408, 536, 768), meta=np.ndarray>

In [218]:
startyear = parse('yearranges').split('|')[0].split(',')[0]
endyear = parse('yearranges').split('|')[0].split(',')[1]
mean_data = arrays[0].sel(time=slice(startyear, endyear)).mean(dim='time')

'BCC'

In [300]:
df_hist = df.query(f"experiment_id == 'historical' & table_id == 'Amon' & variable_id == 'tas' & member_id == 'r1i1p1f1'")
model_match = mean_data.models.values.tolist()
model_match = [model for model in models if model_match in model]
hist_link = df_hist[df_hist['source_id']==model_match[0]]


# get the path to a specific zarr store (the first one from the dataframe above)
zstore = hist_link.zstore.values[-1]
print(zstore)

# create a mutable-mapping-style interface to the store
mapper = fsspec.get_mapper(zstore)

# open it using xarray and zarr
hist_ds = xr.open_zarr(mapper, consolidated=True)

gs://cmip6/CMIP6/CMIP/BCC/BCC-CSM2-MR/historical/r1i1p1f1/Amon/tas/gn/v20181126/


In [308]:
hist_ds_short = hist_ds.sel(time=slice(parse('baselinestart'), parse('baselineend'))).tas.mean(dim='time')
difference = mean_data - hist_ds_short
difference

<xarray.DataArray 'tas' (lat: 160, lon: 320)>
dask.array<sub, shape=(160, 320), dtype=float32, chunksize=(160, 320), chunktype=numpy.ndarray>
Coordinates:
    height   float64 2.0
  * lat      (lat) float64 -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
  * lon      (lon) float64 0.0 1.125 2.25 3.375 4.5 ... 355.5 356.6 357.8 358.9
    models   <U3 'BCC'

In [311]:
map_proj = ccrs.LambertConformal(central_longitude=-95, central_latitude=45)

difference.squeeze().plot()
plt.scatter(float(parse('longitude')),
            float(parse('latitude')),
            facecolors='none',
            edgecolors='black',
            marker='o',
            transform=ccrs.PlateCarree(),
            s=6,
            linewidths=0.5)

ValueError: Axes should be an instance of GeoAxes, got <class 'matplotlib.axes._subplots.AxesSubplot'>

ValueError: Axes should be an instance of GeoAxes, got <class 'matplotlib.axes._subplots.AxesSubplot'>

<Figure size 864x432 with 2 Axes>